### Modeling
- input data
    - keyword(조명, 외장하드, 블루투스 스피커, 물티슈)
    * 쿠팡랭킹 top36까지 크롤링 / top100까지 총 댓글 수 확인
- 전처리
    - 'pos_filtered' == 0인 데이터 삭제
    - filtering 품사 줄이고 stop word 수동 설정

In [212]:
def make_df_train(keyword, item_count_start=0, item_count_end=100):
    import requests
    from scrapy.http import TextResponse
    import json
    
    search_url = "http://www.coupang.com/np/search?q={}&isPriceRange=false&page=1&sorter=scoreDesc&listSize=100".format(keyword)
    rep = requests.get(search_url)
    response = TextResponse(rep.url, body=rep.text, encoding='utf-8')

    # 상품마다의 링크를 불러오기에 앞서 아이템 Top100 리스트 불러오기
    products = json.loads(response.xpath('//*[@id="productList"]/@data-products').extract()[0])['indexes'][item_count_start:item_count_end]
    products = list(set(products))
    print("총 {}개의 상품에서 중복제거 후 {}개의 상품 크롤링".format(item_count_end,len(products)))
    
    df = pd.DataFrame(columns=['rating', "item_nbr", "item", "option", "date", "name", "re_title", 'review']) 

    top100_review_total_cnt = 0
    review_count = 0
    no_review_cnt = 0
    
    for product in products:
        
        # Top100 상품마다의 상품평 개수(나중에 불러오는 정보라 상품링크 안에서는 xpath로 바로 가져올 수 없음)
        try:
            review_count = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
            top100_review_total_cnt += review_count
            
        except Exception as e:
            no_review_cnt += 1
        
        # product for문 안에서 수행하는 코드    
        def make_url(product, review_count):
            page_count = (review_count//100)+1
            review_urls = []
            for page in range(1, page_count+1):
                review_urls.append('http://www.coupang.com/vp/product/reviews?productId={}&page={}&size=100&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=2&ratingSummary=true'.format(product, page))
                # size는 100이 최대로 설정되어 있음
            return review_urls


        # Top100 상품마다의 크롤링할 url(리뷰 크롤링은 '상품평 보기' 클릭 후 나타나는 reviews Request URL에서 해야함)
#         review_urls[product] = make_url(product, review_count)
        review_url = make_url(product, review_count) # 리스트로 반환됨

        for url in review_url:

            review_url = url
            review_rep = requests.get(review_url)
            review_response = TextResponse(review_rep.url, body=review_rep.text, encoding='utf-8')        

            # Top100 상품마다의 상품평 개수
            review_total_cnt = int(review_response.xpath('/html/body/div[2]/@data-review-total-count').extract()[0])

            # Top100 상품마다 한 page의 상품평 개수
            review_url_cnt = int(float(review_response.xpath('count(/html/body/article)').extract()[0]))

            if review_url_cnt < 100:          
                for i in range(1, review_url_cnt+1): # 한개의 review_url에서 100개를 훑고 그 다음 review_url로 넘어가면 됨
                    item_nbr = product

                    date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                    name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                    item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                    if len(item_option_ls) == 1:
                        item = item_option_ls[0]

                    else:    
                        item = item_option_ls[0]
                        option = item_option_ls[1]

                    if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                        re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                        review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                    else:
                        re_title = ['.']
                        review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                    rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                    df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}

            else:
                for i in range(1, 101):
                    item_nbr = product

                    date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                    name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                    item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                    if len(item_option_ls) == 1:
                        item = item_option_ls[0]

                    else:    
                        item = item_option_ls[0]
                        option = item_option_ls[1]

                    if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                        re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                        review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                    else:
                        re_title = ['.']
                        review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                    rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                    df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}

    print("{}개 상품 중 review가 없는 상품 수 : {}".format(item_count_end,no_review_cnt))
    
    def re_title_filter(a):
        for i in range(len(a[0].split('\n'))):
            if len(a[0].split('\n')[i].strip()) != 0:
                return a[0].split('\n')[i].strip()
            else:
                return '.'

    def review_filter(a):
        s = ''
        for i in range(len(a)):
            for ii in range(len(a[i].split('\n'))):
                if a[i].split('\n')[ii].strip() != 0:
                    s += a[i].split('\n')[ii].strip() + ' '
        return s

    def kkma_pos(a):
        from konlpy.tag import Kkma
        kkma = Kkma()
        return kkma.pos(a)

    def kkma_pos_filter(a):
        """
        df['pos']를 받아 품사 필터링을 거쳐 해당 키워드 리스트 반환
        """
        pos_ls = ['NNG','NNP','VA','UN','XR','MAG','ECE']
        ls = []
        s = ""
        for i in range(len(a)):
            #NNG(보통 명사), NNP(고유 명사), VA(형용사), XR(어근), MAG(일반 부사), UN(명사추정범주)
            if a[i][1] in pos_ls:
                s += a[i][0] + ','
        ls.append(s)
        return ls

    def rating_filter(a):
        if a == '3' or a == '4' or a == '5':
            return 1
            # 긍정
        else:
            return 0
            # 부정
    
    df['re_title_filtered'] = df['re_title'].apply(re_title_filter)
    df['review_filtered'] = df['review'].apply(review_filter)
    df['full_review'] = df['re_title_filtered']+df['review_filtered']
    df['pos'] = df['full_review'].apply(kkma_pos)
    df['pos_filtered'] = df['pos'].apply(kkma_pos_filter)
    df['rating_filtered'] = df['rating'].apply(rating_filter)
    
    # 'pos_filtered' == 0인 데이터 삭제
    idx = []
    for i in range(len(df)):
        if len(df['pos_filtered'][i][0]) == 0: # 리스트 안의 문자열의 길이가 0인지를 확인
            idx.append(i)
    df = df.drop(index=idx).reset_index()
    
    print("{}'s Top100 Review Total Count : {}".format(keyword, top100_review_total_cnt))
    print("{}'s Top100 Filtered Review Count : {}".format(keyword, len(df)))
    
    df_fin = df[['rating', 'rating_filtered', 'item_nbr', 'item', 'option', 'date', 'name', 'full_review', 'pos', 'pos_filtered']]
    
    return df_fin

In [213]:
df_train = make_df_train('샘키즈')
df_train.tail()

총 100개의 상품에서 중복제거 후 86개의 상품 크롤링
100개 상품 중 review가 없는 상품 수 : 52
샘키즈's Top100 Review Total Count : 933
샘키즈's Top100 Filtered Review Count : 488


,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
483,5,1,32102900,한샘 샘키즈 3단수납책장 1305 A형 모던,B형 (몸통)크림화이트/(박스)하프그레이(B),2018.08.10,문*경,. 아이장난감정리함으로도 쓰고있는데 너무깔끔하고좋네요~ 배송도빠르고 한샘업체랑연...,"[(., SF), (아이, NNG), (장난감, NNG), (정리, NNG), (하...","[아이,장난감,정리,고,너무,깔끔,고,좋,배송,빠르,고,한샘,업체,연결,기사님,조립..."
484,3,1,32102900,한샘 샘키즈 3단수납책장 1305 A형 모던,B형 (몸통)크림화이트/(박스)하프그레이(B),2017.10.29,김*현,. 맘에드는데 원래 서랍장 안속에 구멍이 많나요? ㅠ 뒤늦게 다 봐서...,"[(., SF), (맘, NNG), (에, JKM), (드, VV), (는데, EC...","[맘,원래,서랍,장,안,속,구멍,많,ㅠ,뒤늦게,다,그냥,ㅠ,]"
485,5,1,32102900,한샘 샘키즈 3단수납책장 1305 A형 모던,A형 (몸통)크림화이트/(박스)올화이트(A),2017.11.10,이*수,. 상품도 깨끗 튼튼하고 배송도 약속한 시간에 와주셨어요.,"[(., SF), (상품, NNG), (도, JX), (깨끗, XR), (튼튼, N...","[상품,깨끗,튼튼,고,배송,약속,시간,]"
486,5,1,32102900,한샘 샘키즈 3단수납책장 1305 A형 모던,B형 (몸통)크림화이트/(박스)하프그레이(B),2018.07.27,최*진,. 왜 진즉에 사지 않았을까 후회중..,"[(., SF), (왜, MAG), (지, VX), (ㄴ즉, ECE), (에, VV...","[왜,ㄴ즉,지,후회,]"
487,3,1,32102900,한샘 샘키즈 3단수납책장 1305 A형 모던,B형 (몸통)크림화이트/(박스)하프그레이(B),2017.11.29,노*윤,. 깔끔하고 수납공간이 많아서 조으네요,"[(., SF), (깔끔, XR), (하, XSA), (고, ECE), (수납, N...","[깔끔,고,수납,공간,많,]"


In [ ]:
# df_train로 만든 단어장
# train 데이터와 test 데이터의 단어장은 동일해야한다

In [ ]:
# https://datascienceschool.net/view-notebook/3387abe1784248339e418835297e17d2/
# from konlpy.tag import Twitter
# pos_tagger = Twitter()

# def tokenize_pos(doc):
#     return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]
# model3 = Pipeline([
#             ('vect', CountVectorizer(tokenizer=tokenize_pos)), 
#             ('mb', MultinomialNB()),
#         ])

In [28]:
ls = []
ls.append(df_train['pos_filtered'][450])
print(ls)

print(list(df_train['pos_filtered'][450]))  # 문자열에 list를 씌우면 한 문자열 안의 글자마다를 list 요소로 만듦

['배송,조금,늦,으나,기사님,후딱,조립,구,조립,스킬,장난,심,생각,서랍,작,한데,산뜻,이쁘,']
['배', '송', ',', '조', '금', ',', '늦', ',', '으', '나', ',', '기', '사', '님', ',', '후', '딱', ',', '조', '립', ',', '구', ',', '조', '립', ',', '스', '킬', ',', '장', '난', ',', '심', ',', '생', '각', ',', '서', '랍', ',', '작', ',', '한', '데', ',', '산', '뜻', ',', '이', '쁘', ',']


In [56]:
# 코퍼스의 형태(문자열들이 하나의 리스트에 담겨있음)
# countvectorizer에 fit하면 문장들을 단어로 쪼갠 vocabulary를 가지고있음을 알 수 있음
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?',    
]
vect = CountVectorizer()
vect.fit(corpus)
display(vect.vocabulary_)

display(vect.transform(['This is the second document.']).toarray())
display(vect.transform(corpus).toarray())

{'and': 0,
 'document': 1,
 'first': 2,
 'is': 3,
 'last': 4,
 'one': 5,
 'second': 6,
 'the': 7,
 'third': 8,
 'this': 9}

array([[0, 1, 0, 1, 0, 0, 1, 1, 0, 1]], dtype=int64)

array([[0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]], dtype=int64)

In [119]:
from sklearn.feature_extraction.text import CountVectorizer
CountVectorizer()

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [120]:
def Vectorizer_1(df_train, pos_filtered_col, df_input, stop_words=None):
    """
    df_fin의 'pos_filtered' column을 받아 vectorized df를 생성해주는 함수
    """
    train_corpus = []
    for i in range(len(df_train[pos_filtered_col])):
        train_corpus.append(df_train[pos_filtered_col][i][0])

    from sklearn.feature_extraction.text import CountVectorizer
    vect = CountVectorizer(token_pattern=r"\b\w+\b", stop_words=stop_words) # 한 글자도 corpus에 포함될 수 있게 해주는 정규표현식
    vect.fit(train_corpus)

    vect_ls = []
    for i in range(len(df_input)):
        vect_ls.append(vect.transform(df_input[pos_filtered_col][i]).toarray()[0])
        # 리스트안의 리스트로 반환되기 때문에 vect_ls 안에 append하기 전에 [0]로 꺼내줌
    
    # 문자열 하나를 문장으로 간주
    # corpus에는 여러 문장이 하나의 리스트 안의 문자열 요소로 들어가야 하고, ['an apple is red', 'the boy is young']
    # transform할때는 한 문장이 하나의 리스트 안의 문자열 요소로 들어가야 함 ['an apple is red']
    
    df_vec = pd.DataFrame(vect_ls)
    
    return df_vec, vect.vocabulary_

In [126]:
%%time
X_train, voca = Vectorizer_1(df_train, 'pos_filtered', df_train, ['고'])
y_train = df_train['rating_filtered']

Wall time: 340 ms


In [127]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model = MultinomialNB()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(classification_report(y_train, y_train_pred))

             precision    recall  f1-score   support

          0       0.90      0.86      0.88        22
          1       0.99      1.00      0.99       467

avg / total       0.99      0.99      0.99       489



In [101]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model = MultinomialNB()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(classification_report(y_train, y_train_pred))

             precision    recall  f1-score   support

          0       1.00      0.86      0.93        22
          1       0.99      1.00      1.00       442

avg / total       0.99      0.99      0.99       464



In [8]:
def make_df_test(link):
    import requests
    from scrapy.http import TextResponse
    import json
    
    input_url = link.split('?')[0]
    product = link.split('?')[0].split("/")[5]
    
    # 상품마다의 링크를 불러오기에 앞서 아이템 Top36 리스트 불러오기
#     products = json.loads(response.xpath('//*[@id="productList"]/@data-products').extract()[0])['indexes'][:36]

    df = pd.DataFrame(columns=['rating', "item_nbr", "item", "option", "date", "name", "re_title", 'review']) 

    reveiw_cnt_url = 'http://www.coupang.com/vp/product/reviews?productId={}&page=1&size=100&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=2&ratingSummary=true'.format(product)
    reveiw_cnt_rep = requests.get(reveiw_cnt_url)
    reveiw_cnt_response = TextResponse(reveiw_cnt_rep.url, body=reveiw_cnt_rep.text, encoding='utf-8')
    
    item_name = reveiw_cnt_response.xpath('/html/body/article[1]/div[1]/div[4]/text()').extract()[0].split(',')[0]
    review_total_cnt = int(reveiw_cnt_response.xpath('/html/body/div[2]/@data-review-total-count').extract()[0])

    def make_url(product, review_total_cnt):
        page_count = (review_total_cnt//100)+1
        review_urls = []
        for page in range(1, page_count+1):
            review_urls.append('http://www.coupang.com/vp/product/reviews?productId={}&page={}&size=100&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=2&ratingSummary=true'.format(product, page))
            # size는 100이 최대로 설정되어 있음
        return review_urls

    review_url = make_url(product, review_total_cnt) # 리스트로 반환됨

    for url in review_url:

        review_url = url
        review_rep = requests.get(review_url)
        review_response = TextResponse(review_rep.url, body=review_rep.text, encoding='utf-8')        

        # Top36 상품마다의 상품평 개수
        review_total_cnt = int(review_response.xpath('/html/body/div[2]/@data-review-total-count').extract()[0])

        # Top36 상품마다 한 page의 상품평 개수
        review_url_cnt = int(float(review_response.xpath('count(/html/body/article)').extract()[0]))

        if review_url_cnt < 100:          
            for i in range(1, review_url_cnt+1): # 한개의 review_url에서 100개를 훑고 그 다음 review_url로 넘어가면 됨
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}

        else:
            for i in range(1, 101):
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}
                    
    def re_title_filter(a):
        for i in range(len(a[0].split('\n'))):
            if len(a[0].split('\n')[i].strip()) != 0:
                return a[0].split('\n')[i].strip()
            else:
                return '.'

    def review_filter(a):
        s = ''
        for i in range(len(a)):
            for ii in range(len(a[i].split('\n'))):
                if a[i].split('\n')[ii].strip() != 0:
                    s += a[i].split('\n')[ii].strip() + ' '
        return s

    def kkma_pos(a):
        from konlpy.tag import Kkma
        kkma = Kkma()
        return kkma.pos(a)

    def kkma_pos_filter(a):
        """
        df['pos']를 받아 품사 필터링을 거쳐 해당 키워드 리스트 반환
        """
        pos_ls = ['NNG','NNP','VA','UN','XR','MAG','ECE']
        ls = []
        s = ""
        for i in range(len(a)):
            #NNG(보통 명사), NNP(고유 명사), VA(형용사), XR(어근), MAG(일반 부사), UN(명사추정범주)
            if a[i][1] in pos_ls:
                s += a[i][0] + ','
        ls.append(s)
        return ls

    def rating_filter(a):
        if a == '3' or a == '4' or a == '5':
            return 1
            # 긍정
        else:
            return 0
            # 부정
    
    df['re_title_filtered'] = df['re_title'].apply(re_title_filter)
    df['review_filtered'] = df['review'].apply(review_filter)
    df['full_review'] = df['re_title_filtered']+df['review_filtered']
    df['pos'] = df['full_review'].apply(kkma_pos)
    df['pos_filtered'] = df['pos'].apply(kkma_pos_filter)
    df['rating_filtered'] = df['rating'].apply(rating_filter)
    
    # 'pos_filtered' == 0인 데이터 삭제
    idx = []
    for i in range(len(df)):
        if len(df['pos_filtered'][i][0]) == 0: # 리스트 안의 문자열의 길이가 0인지를 확인
            idx.append(i)
    df = df.drop(index=idx).reset_index()
    
    print("{}'s Review Total Count : {}".format(item_name, review_total_cnt))
    print("{}'s Filtered Review Count : {}".format(item_name, len(df)))
    
    df_fin = df[['rating', 'rating_filtered', 'item_nbr', 'item', 'option', 'date', 'name', 'full_review', 'pos', 'pos_filtered']]
    
    return df_fin

In [30]:
df_test = make_df_test('http://www.coupang.com/vp/products/32037268?itemId=120733379&vendorItemId=3244199196&q=%EC%83%98%ED%82%A4%EC%A6%88&itemsCount=36&searchId=ad44524f3a1d47bd98abd21fc86d620b&rank=15')
df_test.tail()

한샘 샘키즈 수납장 1005 블라썸 민트_좁은형's Review Total Count : 37
한샘 샘키즈 수납장 1005 블라썸 민트_좁은형's Filtered Review Count : 23


,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
18,5,1,32037268,한샘 샘키즈 수납장 1005 블라썸 민트_좁은형,D형 (몸통)메이플/(박스)민트그레이(D),2018.04.05,신*경,. 대만족,"[(., SF), (대만족, NNG)]","[대만족,]"
19,5,1,32037268,한샘 샘키즈 수납장 1005 블라썸 민트_좁은형,C형 (몸통)크림화이트/(박스)코랄민트(C),2018.03.28,문*희,. 설치해보니 넘이뿌네요,"[(., SF), (설치, NNG), (하, XSV), (어, ECS), (보, V...","[설치,]"
20,4,1,32037268,한샘 샘키즈 수납장 1005 블라썸 민트_좁은형,A형 (몸통)크림화이트/(박스)민트그레이(A),2018.03.06,신*미,. 생각보다 튼튼하고 수납공간 좋아요,"[(., SF), (생각, NNG), (보다, JKM), (튼튼, NNP), (하,...","[생각,튼튼,고,수납,공간,좋,]"
21,5,1,32037268,한샘 샘키즈 수납장 1005 블라썸 민트_좁은형,A형 (몸통)크림화이트/(박스)민트그레이(A),2018.02.12,이*임,. 넘졸아요 수납 잘되요 넉넉해서 ㅎㅎㅎ,"[(., SF), (넘, MAG), (졸, VV), (아요, EFN), (수납, N...","[넘,수납,잘,요,넉넉,]"
22,4,1,32037268,한샘 샘키즈 수납장 1005 블라썸 민트_좁은형,A형 (몸통)크림화이트/(박스)민트그레이(A),2018.01.26,김*희,. 배송 설치 다 만족해요,"[(., SF), (배송, NNG), (설치, NNG), (다, MAG), (만족,...","[배송,설치,다,만족,]"


In [128]:
%%time
X_test, voca = Vectorizer_1(df_train, 'pos_filtered', df_test, stop_words=['고'])
y_test = df_test['rating_filtered']

Wall time: 207 ms


In [129]:
X_test.tail()

,0,1,2,3,4,5,6,7,8,9,...,1461,1462,1463,1464,1465,1466,1467,1468,1469,1470
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [130]:
# model = MultinomialNB()
# model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00         3
          1       1.00      1.00      1.00        20

avg / total       1.00      1.00      1.00        23



In [182]:
def top_word_negative(X_test, y_test_pred, word_cnt=15):
    y_test_pred = pd.DataFrame(y_test_pred, columns=['prediction'])
    pred_n_test = pd.concat([y_test_pred, X_test], axis=1)
    top_word = pd.DataFrame(pred_n_test[pred_n_test['prediction']==0].sum().sort_values(ascending=False)[:word_cnt], columns=['count']).reset_index()
    # reset_index()하면 원래 series에서 갖고있던 index를 value로 만들어줌
    
    top_word['word'] = top_word['index'].apply(match_word)
    top_word['pos'] = top_word['word'].apply(kkma_pos)
    return top_word[['word', 'count']] # 나중에 품사 명사인 것, 의미있는 명사만 필터링해서 보여주도록 개선

In [183]:
top_word_negative(X_test, y_test_pred, word_cnt=10)

,word,count
0,배송,4
1,오후,2
2,레일,2
3,일,2
4,물건,2
5,괜찮,2
6,시간,2
7,로켓,1
8,같,1
9,한샘,1


In [184]:
def top_word_positive(X_test, y_test_pred, word_cnt=15):
    y_test_pred = pd.DataFrame(y_test_pred, columns=['prediction'])
    pred_n_test = pd.concat([y_test_pred, X_test], axis=1)
    top_word = pd.DataFrame(pred_n_test[pred_n_test['prediction']==1].sum().sort_values(ascending=False), columns=['count']).reset_index()
    # reset_index()하면 원래 series에서 갖고있던 index를 value로 만들어줌
    top_word_ = top_word[top_word['index'] != 'prediction'].reset_index(drop=True)
    top_word_ = top_word_[:word_cnt]

    top_word_['word'] = top_word_['index'].apply(match_word)
    top_word_['pos'] = top_word_['word'].apply(kkma_pos)
    return top_word_[['word', 'count']]

In [185]:
top_word_positive(X_test, y_test_pred, word_cnt=10)

,word,count
0,좋,15
1,설치,11
2,기사님,9
3,친절,8
4,수납,7
5,너무,7
6,같,7
7,배송,6
8,많이,5
9,생각,5


In [227]:
def match_word(a):
    for key, value in voca.items():
        if value == a:
            return key
        
def kkma_pos(a):
    from konlpy.tag import Kkma
    kkma = Kkma()
    return kkma.pos(a)

In [61]:
y_test_pred = pd.DataFrame(y_test_pred, columns=['prediction'])
pred_n_test = pd.concat([y_test_pred, X_test], axis=1)

In [62]:
pred_n_test

,prediction,0,1,2,3,4,5,6,7,8,...,1462,1463,1464,1465,1466,1467,1468,1469,1470,1471
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,2,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
top10_word = pd.DataFrame(pred_n_test[pred_n_test['prediction']==0].sum().sort_values(ascending=False)[:20], columns=['count']).reset_index()
# reset_index()하면 원래 series에서 갖고있던 index를 value로 만들어줌
top10_word

,index,count
0,544,4
1,413,2
2,104,2
3,127,2
4,499,2
5,949,2
6,771,2
7,1058,2
8,55,1
9,667,1


In [79]:
def match_word(a):
    for key, value in voca.items():
        if value == a:
            return key
        
def kkma_pos(a):
    from konlpy.tag import Kkma
    kkma = Kkma()
    return kkma.pos(a)

In [99]:
top10_word['word'] = top10_word['index'].apply(match_word)
top10_word['pos'] = top10_word['word'].apply(kkma_pos)

In [103]:
top10_word[['word', 'count']]

,word,count
0,배송,4
1,레일,2
2,고,2
3,괜찮,2
4,물건,2
5,오후,2
6,시간,2
7,일,2
8,같,1
9,상담원,1


In [34]:
ls = list(df_test_return[df_test_return['return']==0].sum().sort_values(ascending=False)[:10].index)
ls

[544, 413, 104, 127, 499, 949, 771, 1058, 55, 667]

In [35]:
keyword = df_test_return[df_test_return['return']==0].sum().sort_values(ascending=False)[:10]
keyword

544     4
413     2
104     2
127     2
499     2
949     2
771     2
1058    2
55      1
667     1
dtype: int64

In [36]:
# [key for key, value in voca.items() if value == ]

for key, value in voca.items():
    for i in ls:
        if value == i:
            print(key, keyword[i]) # 품사 명사인것만 필터링해서보여주자

고 2
배송 4
같 1
시간 2
물건 2
괜찮 2
오후 2
일 2
상담원 1
레일 2


------------------------------------------------------

In [4]:
def review_cnt_check(keyword, item_count_start=0, item_count_end=36):
    import requests
    from scrapy.http import TextResponse
    import json
    search_url = "http://www.coupang.com/np/search?component=&q={}".format(keyword)
    rep = requests.get(search_url)
    response = TextResponse(rep.url, body=rep.text, encoding='utf-8')

    # 상품마다의 링크를 불러오기에 앞서 아이템 Top36 리스트 불러오기
    products = json.loads(response.xpath('//*[@id="productList"]/@data-products').extract()[0])['indexes'][item_count_start:item_count_end]
    
    review_total_cnt = 0
    
    for idx, product in enumerate(products):
        
        # Top36 상품마다의 상품평 개수(나중에 불러오는 정보라 상품링크 안에서는 xpath로 바로 가져올 수 없음)
        try:
            review_count = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
            review_total_cnt += review_count
            print('{}번째 item {}의 review 수 : {}'.format(idx, product, review_count))
            
        except Exception as e:
            print('No reviews')
            
    print('{}번째부터 {}번째 item의 총 review 수 : {}'.format(item_count_start, item_count_end-1, review_total_cnt))

In [6]:
review_cnt_check('조명')

0번째 item 1001428의 review 수 : 4857
1번째 item 36815959의 review 수 : 645
2번째 item 55254805의 review 수 : 15
3번째 item 35863481의 review 수 : 129
4번째 item 36815948의 review 수 : 151
5번째 item 1001428의 review 수 : 4857
6번째 item 36551328의 review 수 : 124
7번째 item 36234235의 review 수 : 684
8번째 item 22678653의 review 수 : 215
9번째 item 16131155의 review 수 : 32
10번째 item 1001428의 review 수 : 4857
11번째 item 27470080의 review 수 : 1437
12번째 item 47836012의 review 수 : 89
13번째 item 4915201의 review 수 : 295
14번째 item 56837841의 review 수 : 5
15번째 item 1001428의 review 수 : 4857
16번째 item 65647114의 review 수 : 94
17번째 item 36234235의 review 수 : 684
18번째 item 23593043의 review 수 : 17
19번째 item 36815959의 review 수 : 645
20번째 item 47836013의 review 수 : 76
21번째 item 44920528의 review 수 : 14
22번째 item 8341176의 review 수 : 390
23번째 item 18042509의 review 수 : 20
24번째 item 36815955의 review 수 : 58
25번째 item 11603277의 review 수 : 83
26번째 item 26447051의 review 수 : 10
27번째 item 34096012의 review 수 : 4
28번째 item 18042509의 review 수 : 20
29번째 item 10

In [190]:
%%time
df_train = make_df_train('조명', 0, 20)
df_train.tail()

20개 상품 중 review가 없는 상품 수 : 0
조명's Top36 Review Total Count : 22896
조명's Top36 Filtered Review Count : 12341
Wall time: 9min 43s


In [20]:
# df_train_concat = pd.concat([df_train, df_train_1]).reset_index(drop=True)
# df_train_concat.tail()

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
13178,5,1,36815959,말랑이 물방울 실리콘 터치 무드등,올망이,2017.12.01,박*국,. 생각이상으로 개안네요,"[(., SF), (생각, NNG), (이상, NNG), (으로, JKM), (개안...","[생각,이상,개안,]"
13179,4,1,36815959,말랑이 물방울 실리콘 터치 무드등,올망이,2017.11.30,M*z,. 조금 작인 해요 ㅠㅠㅠ,"[(., SF), (조금, NNG), (작인, NNG), (하, VV), (어요, ...","[조금,작인,]"
13180,1,0,36815959,말랑이 물방울 실리콘 터치 무드등,올망이,2017.11.28,최*숙,. 사지마세요완전후회가또바로파세요,"[(., SF), (사지, NNG), (마세, NNG), (요, JX), (완전, ...","[사지,마세,완전,후회,또,바로,]"
13181,5,1,36815959,말랑이 물방울 실리콘 터치 무드등,올망이,2017.11.26,장*우,. 컴퓨터랑 같이뒀는데 너무귀엽네오,"[(., SF), (컴퓨터, NNG), (랑, JC), (같이, MAG), (두, ...","[컴퓨터,같이,너무,귀엽,]"
13182,5,1,36815959,말랑이 물방울 실리콘 터치 무드등,졸망이,2017.11.26,장*우,. 컴퓨터랑 같이뒀는데 너무귀여워요,"[(., SF), (컴퓨터, NNG), (랑, JC), (같이, MAG), (두, ...","[컴퓨터,같이,너무,귀엽,]"


In [217]:
X_train, voca = Vectorizer_1(df_train, 'pos_filtered', df_train, ['고'])
y_train = df_train['rating_filtered']

In [218]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model = MultinomialNB()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(classification_report(y_train, y_train_pred))

             precision    recall  f1-score   support

          0       0.90      0.86      0.88        22
          1       0.99      1.00      0.99       466

avg / total       0.99      0.99      0.99       488



In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model = MultinomialNB()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(classification_report(y_train, y_train_pred))

             precision    recall  f1-score   support

          0       0.81      0.66      0.73       344
          1       0.98      0.99      0.99      5982

avg / total       0.97      0.97      0.97      6326



In [195]:
df_test = make_df_test('http://www.coupang.com/vp/products/26431800?itemId=102328458&vendorItemId=3193573519&q=%EC%A1%B0%EB%AA%85&itemsCount=36&searchId=617dada96fdc4b28bc62ab512e2537e9&rank=36')
df_test.tail()

프리즘 충전식 LED 스탠드 PL-1400's Review Total Count : 244
프리즘 충전식 LED 스탠드 PL-1400's Filtered Review Count : 135


,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
130,5,1,26431800,프리즘 충전식 LED 스탠드 PL-1400,단일 색상,2018.02.19,정*영,. 만족합니다.,"[(., SF), (만족, NNG), (하, XSV), (ㅂ니다, EFN), (.,...","[만족,]"
131,5,1,26431800,프리즘 충전식 LED 스탠드 PL-1400,단일 색상,2018.02.05,C*U,. 생각보다 아주 좋네요,"[(., SF), (생각, NNG), (보다, JKM), (아주, MAG), (좋,...","[생각,아주,좋,]"
132,5,1,26431800,프리즘 충전식 LED 스탠드 PL-1400,단일 색상,2017.11.26,강*원,. 좋아요,"[(., SF), (좋, VA), (아요, EFN)]","[좋,]"
133,5,1,26431800,프리즘 충전식 LED 스탠드 PL-1400,단일 색상,2017.10.26,김*희,. 디자인 심플하고 넘 좋아요 완전 강추,"[(., SF), (디, NNG), (자, XSN), (이, VCP), (ㄴ, ET...","[디,심플,고,넘,좋,완전,강추,]"
134,5,1,26431800,프리즘 충전식 LED 스탠드 PL-1400,단일 색상,2017.10.11,변*환,". 크기도 적당하고, 잘 산 거같아요^^","[(., SF), (크기, NNG), (도, JX), (적당, XR), (하, XS...","[크기,적당,고,잘,같,]"


In [197]:
X_test, voca = Vectorizer_1(df_train, 'pos_filtered', df_test)
y_test = df_test['rating_filtered']

In [198]:
# model = MultinomialNB()
# model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          0       0.33      0.25      0.29         4
          1       0.98      0.98      0.98       131

avg / total       0.96      0.96      0.96       135



In [199]:
df_test[df_test["rating_filtered"]==0]

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
12,1,0,26431800,프리즘 충전식 LED 스탠드 PL-1400,단일 색상,2018.02.28,서*훈,. 산지 하루만에 고장났내요...별하나도 아까운듣ㅡㅡ,"[(., SF), (산지, NNG), (하루, NNG), (만, JX), (에, J...","[산지,하루,별하,나,아까,운듣,]"
19,1,0,26431800,프리즘 충전식 LED 스탠드 PL-1400,단일 색상,2018.06.09,심*현,". 가볍고 충전식이고 밝기도 적절해서 좋게 썼는데, 딱 2달 하루에 2시간씩...","[(., SF), (가볍, VA), (고, ECE), (충전, NNG), (식이, ...","[가볍,고,충전,식이,고,밝기,적절,좋,딱,달,하루,시간,고,]"
96,1,0,26431800,프리즘 충전식 LED 스탠드 PL-1400,단일 색상,2018.06.25,박*국,. 쿠팡 로켓배송을 믿지 마세요~~,"[(., SF), (쿠팡, UN), (로켓, NNG), (배송, NNG), (을, ...","[쿠팡,로켓,배송,마,세요,]"
128,2,0,26431800,프리즘 충전식 LED 스탠드 PL-1400,단일 색상,2018.02.27,구*수,. 바로 고장남,"[(., SF), (바로, MAG), (고장나, VV), (ㅁ, ETN)]","[바로,]"


In [200]:
df_test = make_df_test('http://www.coupang.com/vp/products/36234235?itemId=134203412&vendorItemId=3284340606&q=%EC%A1%B0%EB%AA%85&itemsCount=36&searchId=617dada96fdc4b28bc62ab512e2537e9&rank=42')
df_test.tail()

윤우네무드등 하와이안 무드등's Review Total Count : 691
윤우네무드등 하와이안 무드등's Filtered Review Count : 383


,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
378,5,1,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2017.12.06,진*정,. 가격대비 완전 굳이에요 너무이쁨,"[(., SF), (가격대비, NNP), (완전, NNG), (굳이에, UN), (...","[가격대비,완전,굳이에,너무,이쁘,]"
379,5,1,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2017.11.10,오*선,. 완전 굿,"[(., SF), (완전, NNG), (굿, NNG)]","[완전,굿,]"
380,5,1,36234235,윤우네무드등 하와이안 무드등,선인장(GREEN),2017.09.27,구*연,. 아이도좋아라하고이쁘내요,"[(., SF), (아이, NNG), (도, JX), (좋, VA), (아, ECD...","[아이,좋,라,고,이쁘,]"
381,5,1,36234235,윤우네무드등 하와이안 무드등,파인애플(YELLOW),2017.09.27,구*연,. 아이도좋아라하고이쁘내요,"[(., SF), (아이, NNG), (도, JX), (좋, VA), (아, ECD...","[아이,좋,라,고,이쁘,]"
382,5,1,36234235,윤우네무드등 하와이안 무드등,플라밍고(PINK),2017.09.27,구*연,. 아이도좋아라하고 이쁘내요,"[(., SF), (아이, NNG), (도, JX), (좋, VA), (아, ECD...","[아이,좋,라,고,이쁘,]"


In [215]:
X_test, voca = Vectorizer_1(df_train, 'pos_filtered', df_test, stop_words=['고'])
y_test = df_test['rating_filtered']

In [216]:
# model = MultinomialNB()
# model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

ValueError: shapes (383,1471) and (4126,2) not aligned: 1471 (dim 1) != 4126 (dim 0)

In [214]:
top_word_negative(X_test, y_test_pred)

,word,count
0,고,12
1,안,9
2,그냥,7
3,구매,7
4,건전지,7
5,좋,6
6,선인장,6
7,상품,4
8,플라스틱,4
9,느낌,4


In [132]:
df_test = make_df_test('http://www.coupang.com/vp/products/36815948?itemId=135753176&vendorItemId=3289428171&q=%EC%A1%B0%EB%AA%85&itemsCount=36&searchId=c360a27ab190497eb67d5f1e3f67ad5d&rank=3')
df_test.tail()

본진 판타스틱 스타 줄조명 20구's Review Total Count : 155
본진 판타스틱 스타 줄조명 20구's Filtered Review Count : 96


,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
91,5,1,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2018.03.06,박*미,. 예뻐요,"[(., SF), (예쁘, VA), (어요, EFN)]","[예쁘,]"
92,4,1,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2018.02.25,이*혜,. 건전지따로 사야하지만 예뻐요,"[(., SF), (건전지, NNG), (따로, MAG), (사, VV), (아야,...","[건전지,따로,예쁘,]"
93,5,1,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2018.02.03,함*림,. 넘 예쁨~^^,"[(., SF), (넘, MAG), (예쁘, VA), (ㅁ, ETN), (~, SO...","[넘,예쁘,]"
94,5,1,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2018.01.17,김*하,. 좋아요,"[(., SF), (좋, VA), (아요, EFN)]","[좋,]"
95,3,1,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2018.01.01,전*경,. 억~작아요~~건전지 사러가야함 ㅠ. ㄷ,"[(., SF), (억, NR), (~, SO), (작, VA), (아요, EFN)...","[작,건전지,가야,함,ㅠ,ㄷ,]"


In [154]:
df_test = make_df_test('http://www.coupang.com/vp/products/22678653?itemId=88139505&vendorItemId=3151068932&q=%EC%A1%B0%EB%AA%85&itemsCount=36&searchId=c360a27ab190497eb67d5f1e3f67ad5d&rank=8')
df_test.tail()

데이홈 LED 감성 와이어 조명 라인듀's Review Total Count : 215
데이홈 LED 감성 와이어 조명 라인듀's Filtered Review Count : 102


,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
97,5,1,22678653,데이홈 LED 감성 와이어 조명 라인듀,웜화이트 C2,2017.09.05,이*비,. 너무 이뻐요 설치도 간단하네요,"[(., SF), (너무, MAG), (이쁘, VA), (어요, EFN), (설치,...","[너무,이쁘,설치,간단,]"
98,5,1,22678653,데이홈 LED 감성 와이어 조명 라인듀,웜화이트 L2,2017.08.22,강*희,. 완전 맘에 들어요,"[(., SF), (완전, NNG), (맘, NNG), (에, JKM), (듣, V...","[완전,맘,]"
99,5,1,22678653,데이홈 LED 감성 와이어 조명 라인듀,웜화이트 C2,2017.08.08,배*숙,. 방 분위가 완전 달라지고 이뻐요~~^^,"[(., SF), (방, NNG), (분, NNG), (위, NNG), (가, JK...","[방,분,위,완전,고,이쁘,]"
100,5,1,22678653,데이홈 LED 감성 와이어 조명 라인듀,웜화이트 C2,2017.07.28,강*순,. 카프리 먹고 투척 이쁘네요,"[(., SF), (카프리, NNG), (먹, VV), (고, ECE), (투척, ...","[카프리,고,투척,이쁘,]"
101,5,1,22678653,데이홈 LED 감성 와이어 조명 라인듀,웜화이트 C2,2017.07.27,신*주,. ㅜㅜ 병은 없어요,"[(., SF), (ㅜㅜ, EMO), (병, NNG), (은, JX), (없, VA...","[병,없,]"


In [155]:
X_test, voca = Vectorizer_1(df_train_concat, 'pos_filtered', df_test)
y_test = df_test['rating_filtered']

In [156]:
# model = MultinomialNB()
# model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          0       0.50      0.44      0.47         9
          1       0.95      0.96      0.95        93

avg / total       0.91      0.91      0.91       102



In [157]:
y_test_pred = pd.DataFrame(y_test_pred, columns=['return'])
df_test_return = pd.concat([y_test_pred, X_test], axis=1)

In [158]:
list(df_test_return[df_test_return['return']==0].index)

[4, 8, 11, 17, 30, 45, 50, 57]

In [159]:
ls = list(df_test_return[df_test_return['return']==0].index)
df_test[df_test.index.isin(ls)]

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
4,1,0,22678653,데이홈 LED 감성 와이어 조명 라인듀,웜화이트 C2,2018.04.11,김*나,. 이거 구입한지 한달도 안됐는데..불이 안들어와요!! 물건받고 4월 11일날(현...,"[(., SF), (이거, NP), (구입, NNG), (하, XSV), (ㄴ, E...","[구입,달,안되,불이,안,물건,고,현재,건전지,빨리,상품평,건진지,다되,고,오늘,다..."
8,5,1,22678653,데이홈 LED 감성 와이어 조명 라인듀,웜화이트 C2,2017.08.26,이*미,. 받아보고 처음엔 깜깍놀랬어요 무슨 철사만 들어 있고 좀 황당했는데 자세히 ...,"[(., SF), (받으, VV), (아, ECS), (보, VV), (고, ECE...","[고,처음,깜깍놀랬어,철사,고,좀,황,자세히,줄,전구,고,또,병,전원,장치,집,와인..."
11,4,1,22678653,데이홈 LED 감성 와이어 조명 라인듀,웜화이트 C2,2017.11.03,최*라,. 설치해놓으니 집에오는 사람마다 감탄하더라구요! 그러나 구매한 것 중 하나에...,"[(., SF), (설치, NNG), (하, XSV), (어, ECS), (놓, V...","[설치,집,사람,감탄,나,구매,하나,불량,배터리,다르,보다,빛,약하,]"
17,1,0,22678653,데이홈 LED 감성 와이어 조명 라인듀,웜화이트 C2,2018.07.28,김*라,. 건전기 교환 한날 잠시 나오고 안나와여 ㅠ 그냥 일회용 인듯 해요 ㅠ 15 ...,"[(., SF), (걸, VV), (ㄴ, ETD), (전기, NNG), (교환, N...","[전기,교환,날,잠시,고,안,여,ㅠ,그냥,일회용,해,ㅠ,정도,구매,다,일회용,생각,..."
30,1,0,22678653,데이홈 LED 감성 와이어 조명 라인듀,웜화이트 C2,2018.05.02,이*경,. 앵두전구와 술잔 올리는 콜크받침과 함께 주문했는데~~ 이 제품2개를 주문 했...,"[(., SF), (앵두, NNG), (전구, NNG), (와, JKM), (술잔,...","[앵두,전구,술잔,콜크받침,함께,주문,이,제품,주문,안,주문,당시,함께,주문,고,뒤..."
45,5,1,22678653,데이홈 LED 감성 와이어 조명 라인듀,웜화이트 C2,2017.12.09,뽁이,. 코로나 병이없어서 급하게 카프리로.. 이쁘네요,"[(., SF), (코로나, NNG), (병, NNG), (이, JKS), (없, ...","[코로나,병,없,급하,카프리,이쁘,]"
50,5,1,22678653,데이홈 LED 감성 와이어 조명 라인듀,웜화이트 C2,2018.08.06,터닝,. ㅎㅎ맥주병에 넣으려고 열심히 마셔서 공병을 따로 뒀는데 누가 쓰레기인 줄알고 ...,"[(., SF), (ㅎㅎ, EMO), (맥주병, NNG), (에, JKM), (넣,...","[맥주병,열심히,공병,따로,누,쓰레기,고,인생,]"
57,1,0,22678653,데이홈 LED 감성 와이어 조명 라인듀,웜화이트 C2,2018.04.12,최*숙,. 별이 아까워요 이틀만에 3개시켰는데 다 아웃~ 정말 짜증~ 쓰레기에여,"[(., SF), (별, NNG), (이, JKS), (아깝, VA), (어요, E...","[별,아깝,이틀,다,아웃,정말,짜증,쓰레기,]"


In [163]:
ls = list(df_test_return[df_test_return['return']==0].sum().sort_values(ascending=False)[:10].index)
ls

[298, 39, 2413, 3420, 786, 1628, 3042, 1748, 3274, 227]

In [164]:
keyword = df_test_return[df_test_return['return']==0].sum().sort_values(ascending=False)[:10]
keyword

298     10
39       5
2413     4
3420     4
786      3
1628     3
3042     3
1748     2
3274     2
227      2
dtype: int64

In [165]:
# [key for key, value in voca.items() if value == ]

for key, value in voca.items():
    for i in ls:
        if value == i:
            print(key, keyword[i]) # 품사 명사인것만 필터링해서보여주자

고 10
안 4
다 3
주문 4
정도 2
ㅠ 5
불이 2
건전지 2
병 3
일회용 3


In [96]:
review_cnt_check('외장하드', 0, 26)

1번째 item 25271683의 review 수 : 565
2번째 item 4889527의 review 수 : 3023
3번째 item 88785654의 review 수 : 36
4번째 item 60867504의 review 수 : 78
5번째 item 49867193의 review 수 : 259
6번째 item 14820704의 review 수 : 1475
7번째 item 1148353의 review 수 : 2401
8번째 item 57040999의 review 수 : 202
9번째 item 87782199의 review 수 : 55
10번째 item 49867193의 review 수 : 259
11번째 item 14820704의 review 수 : 1475
12번째 item 1148353의 review 수 : 2401
13번째 item 1148365의 review 수 : 1527
14번째 item 87782199의 review 수 : 55
15번째 item 49867193의 review 수 : 259
16번째 item 14820708의 review 수 : 332
17번째 item 1148353의 review 수 : 2401
18번째 item 1148365의 review 수 : 1527
19번째 item 4956527의 review 수 : 1765
20번째 item 14820704의 review 수 : 1475
21번째 item 25271665의 review 수 : 687
22번째 item 1148353의 review 수 : 2401
23번째 item 1148365의 review 수 : 1527
24번째 item 4956527의 review 수 : 1765
25번째 item 14820704의 review 수 : 1475
26번째 item 106697980의 review 수 : 6
1번째부터 26번째 item의 총 review 수 : 29431


In [98]:
%%time
df_train = make_df_train('외장하드', 0, 26)
df_train.tail()

외장하드's Top36 Review Total Count : 29431
외장하드's Top36 Filtered Review Count : 12295
Wall time: 15min 38s


In [99]:
%%time
X_train = Vectorizer(df_train, 'pos_filtered', df_train)
y_train = df_train['rating_filtered']

Wall time: 33.6 s


In [100]:
df_train.tail()

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
12290,5,1,106697980,엠지텍 테란 엣지 케이블 일체형 외장하드 Terran3.1 PMR,1TB,2018.08.23,최*환,. 외장하드 일체형은 처음 사용해보네요 케이블이 망가지면 불편하겠다.. 라는 생...,"[(., SF), (외장, NNG), (하드, NNG), (일체형, NNG), (은...","[외장, 하드, 일체형, 처음, 사용, 케이블, 면, 불편, 생각, 면서, 성비, ..."
12291,5,1,106697980,엠지텍 테란 엣지 케이블 일체형 외장하드 Terran3.1 PMR,1TB,2018.08.23,김*곤,. 역시 배송은 로켓배송 ㅋ 진짜빠르네요~ 받마마자 연결해서 쓰는데 연결하자마자...,"[(., SF), (역시, MAG), (배송, NNG), (은, JX), (로켓, ...","[역시, 배송, 로켓, 배송, ㅋ, 진짜, 빠르, 마자, 연결, 연결, 자마자, 바..."
12292,5,1,106697980,엠지텍 테란 엣지 케이블 일체형 외장하드 Terran3.1 PMR,1TB,2018.08.23,김*진,. 빠른배송 감사합니다. 케이스가 굉장히 견고하고 튼튼합니다! 떨어뜨리면 안되...,"[(., SF), (빠르, VA), (ㄴ, ETD), (배송, NNG), (감사, ...","[빠르, 배송, 감사, 케이스, 굉장히, 견고, 고, 튼튼, 면, 안되, 지만, 끄..."
12293,5,1,106697980,엠지텍 테란 엣지 케이블 일체형 외장하드 Terran3.1 PMR,1TB,2018.08.23,김*수,. 23시30분에 주문했는데도 다음날 바로 오네요 ! 로켓배송 기사님들 정말 수...,"[(., SF), (23, NR), (시, NNM), (30, NR), (분, NN...","[주문, 다음날, 바로, 로켓, 배송, 기사님, 정말, 수고, 많, 다, 그리고, ..."
12294,4,1,106697980,엠지텍 테란 엣지 케이블 일체형 외장하드 Terran3.1 PMR,1TB,2018.08.16,엄*식,. 좋아요 가볍고 선도 일체형이고 다만 뭐가 조잡한 느낌이 듭니다 ㅎㅎ,"[(., SF), (좋, VA), (아요, EFN), (가볍, VA), (고, EC...","[좋, 가볍, 고, 선도, 일체형, 고, 다만, 조잡, 느낌]"


In [101]:
X_train.tail()

,0,1,2,3,4,5,6,7,8,9,...,4410,4411,4412,4413,4414,4415,4416,4417,4418,4419
12290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12293,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model = MultinomialNB()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(classification_report(y_train, y_train_pred))

             precision    recall  f1-score   support

          0       0.81      0.82      0.82       905
          1       0.99      0.98      0.99     11390

avg / total       0.97      0.97      0.97     12295



In [111]:
df_test = make_df_test('http://www.coupang.com/vp/products/1168550?itemId=5040336&vendorItemId=3006743611&q=%EC%99%B8%EC%9E%A5%ED%95%98%EB%93%9C&itemsCount=36&searchId=4b1fbe311eaf480884ffe815a7143a12&rank=36')
df_test.tail()

삼성전자 J3 Portable 외장하드's Review Total Count : 626
삼성전자 J3 Portable 외장하드's Filtered Review Count : 256


,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
251,5,1,1168550,삼성 외장하드 J3,500GB,2016.01.22,조*식,. ㅎ,"[(., SF), (ㅎ, NNG)]",[ㅎ]
252,4,1,1168550,삼성 외장하드 J3,1TB,2016.01.19,김*진,. 삼성은 배신하지 않습니다 ㅎ 잘쓰겠습니다.,"[(., SF), (삼성, NNG), (은, JX), (배신, NNG), (하, X...","[삼성, 배신, ㅎ, 잘]"
253,5,1,1168550,삼성전자 J3 Portable 외장하드,2TB,2016.01.19,이*호,. 배송 굿~ 아직 사용은 안했네요,"[(., SF), (배송, NNG), (굿, NNG), (~, SO), (아직, M...","[배송, 굿, 아직, 사용]"
254,5,1,1168550,삼성전자 J3 Portable 외장하드,2TB,2015.12.02,이*훈,. 디자인도 세련되고 성능도 조아요~~,"[(., SF), (디자인, NNG), (도, JX), (세련, NNG), (되, ...","[디자인, 세련, 고, 성능, 조아]"
255,4,1,1168550,삼성전자 J3 Portable 외장하드,2TB,2015.11.30,이*민,. 전용 케이스가 없는게 다소 아쉽네요.,"[(., SF), (전용, NNG), (케이스, NNG), (가, JKS), (없,...","[전용, 케이스, 없, 다소, 아쉽]"


In [112]:
X_test = Vectorizer(df_train, 'pos_filtered', df_test)
y_test = df_test['rating_filtered']

In [113]:
X_test.tail()

,0,1,2,3,4,5,6,7,8,9,...,4410,4411,4412,4413,4414,4415,4416,4417,4418,4419
251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
252,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
253,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
# model = MultinomialNB()
# model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          0       0.62      0.40      0.48        20
          1       0.95      0.98      0.96       236

avg / total       0.92      0.93      0.93       256



In [ ]:
멀티 스레드 처리
스크래피
안좋은 데이터 불리거나, 좋은 데이터 줄이거나

In [ ]:
# 현재까지의 모든 데이터로 모델링 후
# 기간지정 데이터를 test 데이터로 만들어서 예측
# 상품평마다 긍정 / 부정 예측 후 총 데이터에서 긍정 / 부정 데이터 비중 출력
# 별점 상관없이 상품명 키워드만 보고 긍정 / 부정 예측하는 것이 목표 -> 테스트 데이터 수동으로 라벨링 한 후 다시 report 출력
# 긍정 데이터 주요 키워드 출력, 부정 데이터 주요 키워드 출력
# * 긍정, 부정 데이터 비대칭 해결해야함
# 부정적이지 않은 단어들의 filtering(부분적으로 부정 데이터에만 stop word 적용?)
# 새로운 단어 -> missing value X, 중요한 단어일 수도 있는데 그냥 through된다는 것
# 품사필터링 + stop word 설정으로 모델 업데이트 필요

# countvector 파이프라인 X(함수로 만들어서 적용하면 됨), tf-idf(성능보고 선택적으로)
# kkma 사용자 사전 추가(자바 빌드)

In [ ]:
def Vectorizer(df_train, pos_filtered_col, df_input):
    """
    df_fin의 'pos_filtered' column을 받아 vectorized df를 생성해주는 함수
    """
    train_corpus = []
    for i in range(len(df_train[pos_filtered_col])):
        for ii in range(len(df_train[pos_filtered_col][i])):
            train_corpus.append(df_train[pos_filtered_col][i][ii])
            
    from sklearn.feature_extraction.text import CountVectorizer
    vect = CountVectorizer(token_pattern=r"\b\w+\b") # 한 글자도 corpus에 포함될 수 있게 해주는 정규표현식
    vect.fit(train_corpus)

    vect_ls = []
    for i in range(len(df_input)):
        vect_ls.append((vect.transform(df_input[pos_filtered_col][i]).toarray().sum(axis=0).tolist()))
    
    df_vec = pd.DataFrame(vect_ls)
    return df_vec

In [ ]:
휴리스틱 튜닝
가짜 데이터 추가
stop word도 수동으로
나이브베이즈 parameter - sample weight '으나'

In [40]:
df_test_['pos'].iloc[15]

[('.', 'SF'),
 ('예쁘', 'VA'),
 ('어서', 'ECD'),
 ('구매', 'NNG'),
 ('하', 'XSV'),
 ('었', 'EPT'),
 ('으나', 'ECE'),
 (',', 'SP'),
 ('바닥', 'NNG'),
 ('이', 'JKS'),
 ('비스듬히', 'MAG'),
 ('되', 'VV'),
 ('어서', 'ECD'),
 ('스', 'VV'),
 ('어', 'ECS'),
 ('있', 'VXV'),
 ('기', 'ETN'),
 ('가', 'JKS'),
 ('위태', 'XR'),
 ('롭', 'XSA'),
 ('고', 'ECE'),
 ('자꾸', 'MAG'),
 ('넘어지', 'VV'),
 ('ㅂ니다', 'EFN'),
 ('.', 'SF'),
 ('프', 'UN'),
 ('이', 'VCP'),
 ('라', 'EFN'),
 ('스틱', 'NNG'),
 ('재질', 'NNG'),
 ('이', 'VCP'),
 ('라', 'ECD'),
 ('고급', 'NNG'),
 ('스럽', 'XSA'),
 ('ㅁ', 'ETN'),
 ('과', 'JKM'),
 ('는', 'JX'),
 ('전혀', 'MAG'),
 ('거리', 'NNG'),
 ('가', 'JKS'),
 ('멀', 'VV'),
 ('며', 'ECE'),
 (',', 'SP'),
 ('조잡', 'XR'),
 ('하', 'XSA'),
 ('ㄴ', 'ETD'),
 ('장난감', 'NNG'),
 ('수준', 'NNG'),
 ('의', 'JKG'),
 ('느낌', 'NNG'),
 ('이', 'VCP'),
 ('ㅂ니다', 'EFN'),
 ('.', 'SF'),
 ('가격', 'NNG'),
 ('적', 'XSN'),
 ('으로', 'JKM'),
 ('고급', 'NNG'),
 ('적', 'XSN'),
 ('이', 'VCP'),
 ('ㄴ', 'ETD'),
 ('느낌', 'NNG'),
 ('을', 'JKO'),
 ('바라', 'VV'),
 ('ㄴ', 'ETD'),
 ('것', 'NNB'),
 ('은', 

In [ ]:
kkma-2.0.jar

In [28]:
import requests
from scrapy.http import TextResponse
import json
search_url = "http://www.coupang.com/np/search?component=&q=조명"
rep = requests.get(search_url)
response = TextResponse(rep.url, body=rep.text, encoding='utf-8')

# 상품마다의 링크를 불러오기에 앞서 아이템 Top 10 리스트 불러오기
products = json.loads(response.xpath('//*[@id="productList"]/@data-products').extract()[0])['indexes'][:5]

df = pd.DataFrame(columns=['rating', "item_nbr", "item", "option", "date", "name", "re_title", 'review']) 

review_urls = {}

for product in products:

    # Top 10 상품마다의 상품평 개수(나중에 불러오는 정보라 상품링크 안에서는 xpath로 바로 가져올 수 없음)
    try:
#         review_counts[product] = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
        review_count = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
    except Exception as e:
        print('No reviews')

    def make_url(product, review_count):
        page_count = (review_count//100)+1
        review_urls = []
        for page in range(1, page_count+1):
            review_urls.append('http://www.coupang.com/vp/product/reviews?productId={}&page={}&size=100&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=2&ratingSummary=true'.format(product, page))
            # size는 100이 최대로 설정되어 있음
        return review_urls


    # Top 10 상품마다의 크롤링할 url(리뷰 크롤링은 '상품평 보기' 클릭 후 나타나는 reviews Request URL에서 해야함)
    review_urls[product] = make_url(product, review_count)
    review_url = make_url(product, review_count) # 리스트로 반환됨
#     print(review_url)

    for url in review_url:

        review_url = url
        review_rep = requests.get(review_url)
        review_response = TextResponse(review_rep.url, body=review_rep.text, encoding='utf-8')        

        # Top 10 상품마다의 상품평 개수
        review_total_cnt = int(review_response.xpath('/html/body/div[2]/@data-review-total-count').extract()[0])

        # Top 10 상품마다 한 page의 상품평 개수
        review_url_cnt = int(float(review_response.xpath('count(/html/body/article)').extract()[0]))

        if review_url_cnt < 100:          
            for i in range(1, review_url_cnt+1): # 한개의 review_url에서 100개를 훑고 그 다음 review_url로 넘어가면 됨
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}

        else:
            for i in range(1, 101):
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}

In [29]:
len(df)

5873

In [30]:
df.tail()

,rating,item_nbr,item,option,date,name,re_title,review
5868,3,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.12.26,박*화,[.],[]
5869,5,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.12.26,차*지,[.],[]
5870,5,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.12.20,민또,[.],[]
5871,5,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.12.16,정*현,[.],[]
5872,5,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.11.16,강*희,[.],[]


In [78]:
def re_title_filter(a):
    for i in range(len(a[0].split('\n'))):
        if len(a[0].split('\n')[i].strip()) != 0:
            return a[0].split('\n')[i].strip()
        else:
            return '.'

def review_filter(a):
    s = ''
    for i in range(len(a)):
        for ii in range(len(a[i].split('\n'))):
            if a[i].split('\n')[ii].strip() != 0:
                s += a[i].split('\n')[ii].strip() + ' '
    return s

def kkma_pos(a):
    from konlpy.tag import Kkma
    kkma = Kkma()
    return kkma.pos(a)

def kkma_pos_filter(a):
    """
    df['pos']를 받아 품사 필터링을 거쳐 해당 키워드 리스트 반환
    """
    ls = []
    for i in range(len(a)):
        #NNG(보통 명사), NNP(고유 명사), VA(형용사), XR(어근), MAG(일반 부사), UN(명사추정범주)
        if a[i][1]=='NNG' or a[i][1]=='NNP' or a[i][1]=='VA' or a[i][1]=='UN' or a[i][1]=='XR' or a[i][1]=='MAG':
            ls.append(a[i][0])
    return ls

def rating_filter(a):
    if a == '4' or a == '5':
        return 1
        # 긍정
    else:
        return 0
        # 부정

In [79]:
df_ = df[253:255]
df_['re_title_filtered'] = df_['re_title'].apply(re_title_filter)
df_['review_filtered'] = df_['review'].apply(review_filter)
df_['full_review'] = df_['re_title_filtered']+df_['review_filtered']
df_['pos'] = df_['full_review'].apply(kkma_pos)
df_['pos_filtered'] = df_['pos'].apply(kkma_pos_filter)
df_['rating_filtered'] = df_['rating'].apply(rating_filter)

df_.tail(3)
# df_fin = df_[['rating', 'rating_filtered', 'item_nbr', 'item', 'option', 'date', 'name', 'full_review', 'pos', 'pos_filtered']]
# df_fin.tail(3)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,rating,item_nbr,item,option,date,name,re_title,review,re_title_filtered,review_filtered,full_review,pos,pos_filtered,rating_filtered
253,5,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2018.04.11,박*기,[\n \n \n ...,[\n \n ...,.,너무 이뻐요ㅠㅠㅠㅠ진짜 사길잘했다는 생각 가격도 저렴하고 혼자서도 척척 잘조립되...,. 너무 이뻐요ㅠㅠㅠㅠ진짜 사길잘했다는 생각 가격도 저렴하고 혼자서도 척척 잘조립...,"[(., SF), (너무, MAG), (이쁘, VA), (어요, EFN), (ㅠㅠㅠ...","[너무, 이쁘, 진짜, 길, 생각, 가격, 저렴, 혼자, 척척, 잘, 조립, 그냥,...",1
254,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2018.02.09,박*서,[\n \n 배송상태 ...,[\n \n ...,.,구멍이란 구멍엔 전부 스트로폼이 잔뜩 꼈군요. 제거하느라 면봉이며 이쑤시개며 고...,. 구멍이란 구멍엔 전부 스트로폼이 잔뜩 꼈군요. 제거하느라 면봉이며 이쑤시개며 ...,"[(., SF), (구멍, NNG), (이, VCP), (란, ETD), (구멍, ...","[구멍, 구멍, 전부, 스트로, 폼, 잔뜩, 제거, 면봉, 이쑤시개, 고생, 평, ...",0


In [61]:
df_

,rating,item_nbr,item,option,date,name,re_title,review,re_title_filtered,review_filtered,full_review
253,5,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2018.04.11,박*기,[\n \n \n ...,[\n \n ...,None,너무 이뻐요ㅠㅠㅠㅠ진짜 사길잘했다는 생각 가격도 저렴하고 혼자서도 척척 잘조립되...,NaN
254,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2018.02.09,박*서,[\n \n 배송상태 ...,[\n \n ...,"배송상태 불량 200%, 제품상태 -200%",구멍이란 구멍엔 전부 스트로폼이 잔뜩 꼈군요. 제거하느라 면봉이며 이쑤시개며 고...,"배송상태 불량 200%, 제품상태 -200% 구멍이란 구멍엔 전부 스트로폼이 잔뜩..."


In [62]:
df_['re_title']

253    [\n                \n                     \n  ...
254    [\n                \n                    배송상태 ...
Name: re_title, dtype: object

In [77]:
df_['re_title'][254][0].split('\n')

for i in range(len(df_['re_title'][254][0].split('\n'))):
    if len(df_['re_title'][254][0].split('\n')[i].strip()) != 0:
        print(df_['re_title'][254][0].split('\n')[i].strip(), type(df_['re_title'][254][0].split('\n')[i].strip()))
    else:
        print('none')

none
none
배송상태 불량 200%, 제품상태 -200% <class 'str'>
none
none


In [ ]:
def re_title_filter(a):
    for i in range(len(a[0].split('\n'))):
        if len(a[0].split('\n')[i].strip()) != 0:
            return a[0].split('\n')[i].strip()